In [1]:
import pandas as pd
import tweepy
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
import string

In [2]:
#loads the CSV file and selects only the column relevant to this analysis
df = pd.read_csv("covidvaccinedata.csv")


C:\Users\schoo2\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6,7,8,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df

,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,1,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64.0,11,110,FALSE,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
1,2,Shubham Gupta,NaN,I will tell about all experiences of my life f...,14-08-2020 16:42,1.0,17,0,FALSE,18-08-2020 12:55,#CoronavirusVaccine #CoronaVaccine #CovidVacci...,"['CoronavirusVaccine', 'CoronaVaccine', 'Covid...",Twitter for Android,False
2,3,Journal of Infectiology,NaN,Journal of Infectiology (ISSN 2689-9981) is ac...,14-12-2017 07:07,143.0,566,8,FALSE,18-08-2020 12:46,Deaths due to COVID-19 in Affected Countries\n...,NaN,Twitter Web App,False
3,4,Zane,NaN,Fresher than you.,18-09-2019 11:01,29.0,25,620,FALSE,18-08-2020 12:45,@Team_Subhashree @subhashreesotwe @iamrajchoco...,NaN,Twitter for Android,False
4,5,Ann-Maree O’Connor,"Adelaide, South Australia",Retired university administrator. Melburnian b...,24-01-2013 14:53,83.0,497,10737,FALSE,18-08-2020 12:45,@michellegrattan @ConversationEDU This is what...,NaN,Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156491,156492,Andreas Rechkemmer,"Doha, Qatar",Professor of Global Policy and Founding Direct...,4/8/2016 14:59,130.0,427,60,False,3/4/2021 12:19,Very good news about the efficacy of the J&amp...,"['COVID19', 'COVIDVaccination', 'CovidVaccine']",Twitter for iPhone,False
156492,156493,Dr. Akbar Niazi Teaching Hospital,NaN,Dr. Akbar Niazi Teaching Hospital (ANTH) is a ...,3/24/2019 7:32,83.0,9,1,False,3/4/2021 12:18,🇨🇳's Foreign Minister Wang Yi has also said th...,NaN,Twitter Web App,False
156493,156494,Bnar Talabani,UK,@gw4_cat Wellcome Trust Fellow. Clinical Acade...,3/11/2009 17:32,327.0,909,593,False,3/4/2021 12:18,Super excited to have received my second #covi...,['covidvaccine'],Twitter for iPhone,False
156494,156495,Sophia,NaN,'One day the war will end and I will resume my...,6/2/2010 2:04,6688.0,2744,64605,False,3/4/2021 12:18,"Repeat after me, Canadians are polite, they wo...",['Trudeau'],Twitter Web App,False


In [3]:
# drop the NaNs in the 'text' column

df3 = df.dropna(subset=['text'])

In [4]:
# identify the ones that are NaN

df3[df3['text'].isna() == True]

,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet


In [6]:
# validate the data types in the dataset

df3['text'].apply(lambda x : type(x)).unique()

array([<class 'str'>], dtype=object)

In [7]:
df3

,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,1,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64.0,11,110,FALSE,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False
1,2,Shubham Gupta,NaN,I will tell about all experiences of my life f...,14-08-2020 16:42,1.0,17,0,FALSE,18-08-2020 12:55,#CoronavirusVaccine #CoronaVaccine #CovidVacci...,"['CoronavirusVaccine', 'CoronaVaccine', 'Covid...",Twitter for Android,False
2,3,Journal of Infectiology,NaN,Journal of Infectiology (ISSN 2689-9981) is ac...,14-12-2017 07:07,143.0,566,8,FALSE,18-08-2020 12:46,Deaths due to COVID-19 in Affected Countries\n...,NaN,Twitter Web App,False
3,4,Zane,NaN,Fresher than you.,18-09-2019 11:01,29.0,25,620,FALSE,18-08-2020 12:45,@Team_Subhashree @subhashreesotwe @iamrajchoco...,NaN,Twitter for Android,False
4,5,Ann-Maree O’Connor,"Adelaide, South Australia",Retired university administrator. Melburnian b...,24-01-2013 14:53,83.0,497,10737,FALSE,18-08-2020 12:45,@michellegrattan @ConversationEDU This is what...,NaN,Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156491,156492,Andreas Rechkemmer,"Doha, Qatar",Professor of Global Policy and Founding Direct...,4/8/2016 14:59,130.0,427,60,False,3/4/2021 12:19,Very good news about the efficacy of the J&amp...,"['COVID19', 'COVIDVaccination', 'CovidVaccine']",Twitter for iPhone,False
156492,156493,Dr. Akbar Niazi Teaching Hospital,NaN,Dr. Akbar Niazi Teaching Hospital (ANTH) is a ...,3/24/2019 7:32,83.0,9,1,False,3/4/2021 12:18,🇨🇳's Foreign Minister Wang Yi has also said th...,NaN,Twitter Web App,False
156493,156494,Bnar Talabani,UK,@gw4_cat Wellcome Trust Fellow. Clinical Acade...,3/11/2009 17:32,327.0,909,593,False,3/4/2021 12:18,Super excited to have received my second #covi...,['covidvaccine'],Twitter for iPhone,False
156494,156495,Sophia,NaN,'One day the war will end and I will resume my...,6/2/2010 2:04,6688.0,2744,64605,False,3/4/2021 12:18,"Repeat after me, Canadians are polite, they wo...",['Trudeau'],Twitter Web App,False


In [5]:
def cleanUpTweet(txt):
    # Remove mentions
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#', '', txt)
    # Remove retweets:
    txt = re.sub(r'RT : ', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt) 
    #removes stop words
    txt = re.sub(r'the', '', txt)
    txt = re.sub(r'and', '', txt)
    txt = re.sub(r'to', '', txt)
    
    #remove punctuations 
    txt = re.sub(r'[^\w\s]', '', txt)
    
    return txt

df3['text'] = df3['text'].apply(cleanUpTweet)

<ipython-input-5-fd251c5d9018>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['text'] = df3['text'].apply(cleanUpTweet)


In [6]:
df3

,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,1,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64.0,11,110,FALSE,18-08-2020 12:55,Australia Manufacture Covid19 Vaccine give i...,['CovidVaccine'],Twitter Web App,False
1,2,Shubham Gupta,NaN,I will tell about all experiences of my life f...,14-08-2020 16:42,1.0,17,0,FALSE,18-08-2020 12:55,CoronavirusVaccine CoronaVaccine CovidVaccine ...,"['CoronavirusVaccine', 'CoronaVaccine', 'Covid...",Twitter for Android,False
2,3,Journal of Infectiology,NaN,Journal of Infectiology (ISSN 2689-9981) is ac...,14-12-2017 07:07,143.0,566,8,FALSE,18-08-2020 12:46,Deaths due COVID19 in Affected Countries\nRea...,NaN,Twitter Web App,False
3,4,Zane,NaN,Fresher than you.,18-09-2019 11:01,29.0,25,620,FALSE,18-08-2020 12:45,Stay safe di amp da,NaN,Twitter for Android,False
4,5,Ann-Maree O’Connor,"Adelaide, South Australia",Retired university administrator. Melburnian b...,24-01-2013 14:53,83.0,497,10737,FALSE,18-08-2020 12:45,This is what passes for leadership in our co...,NaN,Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156491,156492,Andreas Rechkemmer,"Doha, Qatar",Professor of Global Policy and Founding Direct...,4/8/2016 14:59,130.0,427,60,False,3/4/2021 12:19,Very good news about efficacy of JampJ vacci...,"['COVID19', 'COVIDVaccination', 'CovidVaccine']",Twitter for iPhone,False
156492,156493,Dr. Akbar Niazi Teaching Hospital,NaN,Dr. Akbar Niazi Teaching Hospital (ANTH) is a ...,3/24/2019 7:32,83.0,9,1,False,3/4/2021 12:18,s Foreign Minister Wang Yi has also said that ...,NaN,Twitter Web App,False
156493,156494,Bnar Talabani,UK,@gw4_cat Wellcome Trust Fellow. Clinical Acade...,3/11/2009 17:32,327.0,909,593,False,3/4/2021 12:18,Super excited have received my second covidva...,['covidvaccine'],Twitter for iPhone,False
156494,156495,Sophia,NaN,'One day the war will end and I will resume my...,6/2/2010 2:04,6688.0,2744,64605,False,3/4/2021 12:18,Repeat after me Canadians are polite y wont lo...,['Trudeau'],Twitter Web App,False


In [9]:
df3.to_csv (r'C:\Users\schoo2\Desktop\HW\Covid-Vaccine-Hashtagged-Tweets\Cleaned_Tweets_030821.csv', index = False, header = True, sep = "|")